# Working with Code

In [16]:
from dotenv import load_dotenv
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
import os

load_dotenv()
hugging_face_api = os.environ["HUGGINGFACEHUB_API_TOKEN"]
repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(
    repo_id=repo_id,  model_kwargs={"temperature": 0.2, "max_new_tokens": 1000}
)

/var/folders/bb/2qsf8cm95jg05zrms2_8m2qr0000gn/T/ipykernel_83681/3371940810.py:9: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [17]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
repo_id = "bigcode/starcoderplus"
llm = HuggingFaceHub(
    repo_id=repo_id,  model_kwargs={"temperature": 0.2, "max_new_tokens": 500}
)

In [18]:
from datasets import load_dataset

# Reload without cache
dataset = load_dataset("imdb", trust_remote_code=True)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [19]:
dataset["train"][100]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)




















































Map: 100%|██████████| 50000/50000 [00:10<00:00, 4942.95 examples/s]


In [22]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets['train'][100]['input_ids']

[101,
 12008,
 27788,
 2523,
 119,
 151,
 9435,
 11455,
 119,
 133,
 9304,
 120,
 135,
 133,
 9304,
 120,
 135,
 1636,
 12058,
 1132,
 2066,
 17355,
 9860,
 119,
 1109,
 2523,
 1108,
 2213,
 119,
 2009,
 146,
 1138,
 1106,
 7380,
 1113,
 1115,
 146,
 1274,
 112,
 189,
 1221,
 119,
 1188,
 1110,
 1640,
 170,
 5671,
 1104,
 1139,
 1159,
 119,
 146,
 1198,
 1458,
 1106,
 11857,
 1639,
 119,
 138,
 6005,
 2386,
 1142,
 2523,
 119,
 1109,
 3176,
 22797,
 1105,
 1103,
 2269,
 1110,
 1198,
 182,
 14824,
 7770,
 119,
 6304,
 1107,
 1451,
 1236,
 119,
 1109,
 1178,
 3505,
 1645,
 1164,
 1103,
 2523,
 1132,
 14760,
 9368,
 138,
 19610,
 2315,
 112,
 188,
 13016,
 119,
 2431,
 1115,
 1108,
 9832,
 1463,
 1118,
 170,
 6434,
 1105,
 8362,
 23063,
 4902,
 9372,
 2741,
 119,
 1109,
 2523,
 1110,
 170,
 9874,
 14255,
 19091,
 5790,
 1105,
 5733,
 8362,
 26438,
 2727,
 1104,
 14946,
 119,
 133,
 9304,
 120,
 135,
 133,
 9304,
 120,
 135,
 10899,
 1208,
 146,
 1821,
 1198,
 1280,
 1106,
 26133,
 1113,
 

In [23]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

In [24]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PyTorch Version: 2.2.2
CUDA Available: False


In [25]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

Using the latest cached version of the module from /Users/maukanmir/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Tue Dec 31 11:59:28 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs = 2, evaluation_strategy="epoch")

/Users/maukanmir/miniforge3/envs/openAI/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## What is fine-tuning?

#### Fine-tuning is a technique of transfer learning in which the weights of a pretrained neural network are used as the initial values for training a new neural network on a different task. This can improve the performance of the new network by leveraging the knowledge learned from the previous task, especially when the new task has limited data.


#### Transfer learning is a technique in machine learning that involves using the knowledge learned from one task to improve the performance on a related but different task. For example, if you have a model that can recognize cars, you can use some of its features to help you recognize trucks. Transfer learning can save you time and resources by reusing existing models instead of training new ones from scratch.

#### Imagine you want to train a computer vision neural network to recognize different types of flowers, such as roses, sunflowers, and tulips. You have a lot of photos of flowers, but not enough to train a model from scratch.

#### Instead, you can use transfer learning, which means taking a model that was already trained on a different task and using some of its knowledge for your new task. For example, you can take a model that was trained to recognize many vehicles, such as cars, trucks, and bicycles. This model has learned how to extract features from images, such as edges, shapes, colors, and textures. These features are useful for any image recognition task, not just the original one.

#### You can use this model as a base for your flower recognition model. You only need to add a new layer on top of it, which will learn how to classify the features into flower types. This layer is called the classifier layer, and it is needed for the model to adapt to the new task. Training the classifier layer on top of the base model is a process called feature extraction. Once this step is done, you can further tailor your model with fine-tuning by unfreezing some of the base model layers and training them together with the classifier layer. This allows you to adjust the base model features to better suit your task.

#### Fine-tuning is usually done after feature extraction, as a final step to improve the performance of the model. You can decide how many layers to unfreeze based on your data size and complexity. A common practice is to unfreeze the last few layers of the base model, which are more specific to the original task, and leave the first few layers frozen, which are more generic and reusable.

#### To summarize, transfer learning and fine-tuning are techniques that allow you to use a pretrained model for a new task. Transfer learning involves adding a new classifier layer on top of the base model and training only that layer. Fine-tuning involves unfreezing some or all of the base model layers and training them together with the classifier layer.

#### In the context of generative AI, fine-tuning is the process of adapting a pretrained language model to a specific task or domain by updating its parameters on a task-specific dataset. Fine-tuning can improve the performance and accuracy of the model for the target task. The steps involved in fine-tuning are:

#### Load the pretrained language model and its tokenizer: The tokenizer is used to convert text into numerical tokens that the model can process. Different models have unique architectures and requirements, often coming with their own specialized tokenizers designed to handle their specific input formats.
#### For instance, BERT (which stands for Bidirectional Encoder Representations from Transformers) uses WordPiece tokenization, while GPT-2 employs byte-pair encoding (BPE). Models also impose token limits due to memory constraints during training and inference.

#### These limits determine the maximum sequence length that a model can handle. For example, BERT has a maximum token limit of 512 tokens, while the GPT-2 can handle longer sequences (e.g., up to 1,024 tokens).

#### Prepare the task-specific dataset: The dataset should contain input-output pairs that are relevant to the task. For example, for sentiment analysis, the input could be a text review and the output could be a sentiment label (positive, negative, or neutral).
#### Define the task-specific head: The head is a layer or a set of layers that are added on top of the pretrained model to perform the task. The head should match the output format and size of the task. For example, for sentiment analysis, the head could be a linear layer with three output units corresponding to the three sentiment labels.


#### When dealing with an LLM specifically designed for text generation, the architecture differs from models used for classification or other tasks. In fact, unlike classification tasks, where we predict labels, an LLM predicts the next word or token in a sequence. This layer is added on top of the pretrained transformer-based models with the purpose of transforming the contextualized hidden representations from the base model into probabilities over the vocabulary.

#### Train the model on the task-specific dataset: The training process involves feeding the input tokens to the model, computing the loss between the model output and the true output, and updating the model parameters using an optimizer. The training can be done for a fixed number of epochs or until a certain criterion is met.
#### Evaluate the model on a test or validation set: The evaluation process involves measuring the performance of the model on unseen data using appropriate metrics. For example, for sentiment analysis, the metric could be accuracy or F1-score (which will be discussed later in this chapter). The evaluation results can be used to compare different models or fine-tuning strategies.
#### Even though it is less computationally and time expensive than full training, fine-tuning an LLM is not a “light” activity. As LLMs are, by definition, large, their fine-tuning has hardware requirements as well as data collection and preprocessing.

#### So, the first question that you want to ask yourself while approaching a given scenario is: “Do I really need to finetune my LLM?”

## When is fine-tuning necessary?
#### As we saw in previous chapters, good prompt engineering combined with the non-parametric knowledge you can add to your model via embeddings are exceptional techniques to customize your LLM, and they can account for around 90% of use cases. However, the preceding affirmation tends to hold for the state-of-the-art models, such as GPT-4, Llama 2, and PaLM 2. As discussed, those models have a huge number of parameters that make them heavy, hence the need for computational power; plus, they might be proprietary and subject to a pay-per-use cost.

#### Henceforth, fine-tuning might also be useful when you want to leverage a light and free-of-charge LLM, such as the Falcon LLM 7B, yet you want it to perform as well as a SOTA model in your specific task.

#### Some examples of when fine-tuning might be necessary are:

- When you want to use an LLM for sentiment analysis on movie reviews, but the LLM was pretrained on Wikipedia articles and books. Fine-tuning can help the LLM learn the vocabulary, style, and tone of movie reviews, as well as the relevant features for sentiment classification.
- When you want to use an LLM for text summarization on news articles, but the LLM was pretrained on a language modeling objective. Fine-tuning can help the LLM learn the structure, content, and length of summaries, as well as the generation objective and evaluation metrics.
- When you want to use an LLM for machine translation between two languages, but the LLM was pretrained on a multilingual corpus that does not include those languages. Fine-tuning can help the LLM learn the vocabulary, grammar, and syntax of the target languages, as well as the translation objective and alignment methods.
- When you want to use an LLM to perform complex named entity recognition (NER) tasks. For example, financial and legal documents contain specialized terminology and entities that are not typically prioritized in general language models, henceforth a fine-tuning process might be extremely beneficial here.


## The BERT model has two main components:

- Encoder: The encoder consists of multiple layers of transformer blocks, each with a self-attention layer and a feedforward layer. The encoder takes as input a sequence of tokens, which are the basic units of text, and outputs a sequence of hidden states, which are high-dimensional vectors that represent the semantic information of each token.
- Output layer: The output layer is task-specific and can be different depending on the type of task that BERT is used for. For example, for text classification, the output layer can be a linear layer that predicts the class label of the input text. For question answering, the output layer can be two linear layers that predict the start and end positions of the answer span in the input text.
- The number of layers and parameters of the model depends on the model version. In fact, BERT comes in two sizes: BERTbase and BERTlarge.